In [48]:
# Connect to openai 

import openai
import os
import time

openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()
assistant = client.beta.assistants.retrieve("asst_kCSrKaHjh589gbKr2fphQ93T")


# Open the file and read its contents
with open("message_thread.txt", 'r', encoding='utf-8') as file:
    message_to_send = file.read()
    
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=message_to_send
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

def check_run_status(client, thread_id, run_id):
    try:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        return run.status  # Access the status attribute directly
    except Exception as e:
        print(f"Error retrieving run: {e}")
        return None

def wait_for_run_completion(client, thread_id, run_id, interval=0.5):
    while True:
        status = check_run_status(client, thread_id, run_id)
        if status == 'completed':
            print("Run is completed.")
            break
        elif status in ['failed', 'cancelled']:
            print(f"Run {status}.")
            break
        else:
            print(f"Run is still in progress... Current status: {status}")
            time.sleep(interval)


# Get the thread ID and run ID
thread_id = thread.id
run_id = run.id

print(f"Thread ID: {thread_id}")
print(f"Run ID: {run_id}")
# print the separator
print("-" * 20)

# Wait for the run to complete
wait_for_run_completion(client, thread_id, run_id)

# print the separator
print("-" * 20)

messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

# Variable to store the last assistant's response
last_assistant_response = ""

for message in messages.data:
    # Check if the message is from the assistant and has content
    if message.role == "assistant" and message.content:
        # Assuming the first item in content is a text object
        text_content = message.content[0].text
        if text_content:
            # Print and store the message text value
            last_assistant_response = text_content.value
            print(f"Assistant: {last_assistant_response}")  # Print the value

Thread ID: thread_P2fVLo2pX8NOTs5YQB7d37sI
Run ID: run_UDaVBqc4UDwTYX20XgLN96N8
--------------------
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is still in progress... Current status: in_progress
Run is completed.
--------------------
Assistant: Клиент хочет тур в Дубай на 1 день на новый год, чтобы посмотреть фейерверк.

- Направление: Дубай
- Количество взрослых: 1
- Дети: Нет
- Нужен ли прямой трансфер из Москвы в отель в Дубае
- Предпочтение по аэропорту вылета: Нет
- Предпочтение по обратной связи: телефон

Tips для менеджера: Уточнить детали о прямом трансфере из Москвы в отель и предложить оптимальные варианты для просмотра фейерверка.


In [50]:
message_to_send

'message_to_send = "User Привет ИИ - Консьерж Здравствуйте! Как я могу помочь вам сегодня? User Мне нужно поехать в дубай ИИ - Консьерж Отличный выбор! Дубай - замечательное место для путешествия. Это будет тур или вы заинтересованы в бронировании отдельных услуг?\n\nUser Тур\n\nИИ - Консьерж Прекрасно! Для начала, могли бы вы уточнить, на какие даты вы планируете поездку в Дубай? И насколько долго вы хотели бы там остаться?\n\nUser на 1 день на новый год, мне нравится стоять в пробке чтобы посмотреть фейерверк\n\nИИ - Консьерж Поездка в Дубай на Новый год звучит восхитительно! Для более точного подбора тура мне нужно знать, сколько вас будет взрослых и есть ли среди путешественников дети? Если есть дети, сообщите, пожалуйста, их возраст.\n\nUser 1 взрослый\n\nИИ - Консьерж Отлично, спасибо за информацию. Могу я узнать, предпочитаете ли вы прямой трансфер в отель после прибытия в Дубай?\n\nUser прямой трансфер из москвы, отель без разницы какой, нужно такси до отеля\n\nИИ - Консьерж По

# API work

FastAPI

## Chaetbot API part
- imput we will receive the **user_id**, **thread_id** and the initial **message**
    - output we will send the message to the user and the **thread_id**
Here we will use cahtconsierge assistant asst_CjvyFIeraCLKB8NTAqF0FhqG

## Sumariser part
- input we will receive the **message** and the **thread_id**
    - output we will send the **summary** and the **thread_id**

here we will use the summariser assistant asst_kCSrKaHjh589gbKr2fphQ93T